In [45]:
import pandas as pd
import numpy as np
import os

# Set the random seed
np.random.seed(42)


In [46]:
data=pd.read_csv('malware.csv', sep ="|")
data.head()

,Name,md5,Machine,SizeOfOptionalHeader,Characteristics,MajorLinkerVersion,MinorLinkerVersion,SizeOfCode,SizeOfInitializedData,SizeOfUninitializedData,...,ResourcesNb,ResourcesMeanEntropy,ResourcesMinEntropy,ResourcesMaxEntropy,ResourcesMeanSize,ResourcesMinSize,ResourcesMaxSize,LoadConfigurationSize,VersionInformationSize,legitimate
0,memtest.exe,631ea355665f28d4707448e442fbf5b8,332,224,258,9,0,361984,115712,0,...,4,3.262823,2.568844,3.537939,8797.000000,216,18032,0,16,1
1,ose.exe,9d10f99a6712e28f8acd5641e3a7ea6b,332,224,3330,9,0,130560,19968,0,...,2,4.250461,3.420744,5.080177,837.000000,518,1156,72,18,1
2,setup.exe,4d92f518527353c0db88a70fddcfd390,332,224,3330,9,0,517120,621568,0,...,11,4.426324,2.846449,5.271813,31102.272727,104,270376,72,18,1
3,DW20.EXE,a41e524f8d45f0074fd07805ff0c9b12,332,224,258,9,0,585728,369152,0,...,10,4.364291,2.669314,6.400720,1457.000000,90,4264,72,18,1
4,dwtrig20.exe,c87e561258f2f8650cef999bf643a731,332,224,258,9,0,294912,247296,0,...,2,4.306100,3.421598,5.190603,1074.500000,849,1300,72,18,1


In [47]:
import pandas as pd

def determine_column_type(df):
    categorical_columns = []
    continuous_columns = []

    for column in df.columns:
        if df[column].dtype == 'object':
            categorical_columns.append(column)
        elif pd.api.types.is_numeric_dtype(df[column]):
            unique_values = df[column].nunique()
            if unique_values < 10:  # Adjust this threshold as needed
                categorical_columns.append(column)
            else:
                continuous_columns.append(column)
        else:
            # Handle other data types if necessary
            pass

    return categorical_columns, continuous_columns



categorical_columns, continuous_columns = determine_column_type(data)

df_categorical = data[categorical_columns]
df_continuous = data[continuous_columns]


In [48]:
y = df_categorical['legitimate']

# scale data

In [49]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the continuous DataFrame
df_continuous_scaled = pd.DataFrame(scaler.fit_transform(df_continuous), columns=df_continuous.columns)


In [50]:
df_continuous_scaled.to_csv("df_continuous_scaled.csv")
y.to_csv("y.csv")

# split data

In [51]:
from sklearn.model_selection import train_test_split

# Split the data into training and temporary sets (80% training, 20% temporary)
X_train, X_temp, y_train, y_temp = train_test_split(df_continuous_scaled, y, test_size=0.2, random_state=42)

# Split the temporary set into validation and testing sets (50% validation, 50% testing)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print("Training set size:", X_train.shape)
print("Validation set size:", X_val.shape)
print("Testing set size:", X_test.shape)

Training set size: (110437, 49)
Validation set size: (13805, 49)
Testing set size: (13805, 49)


In [52]:
y_train.value_counts()

0    77474
1    32963
Name: legitimate, dtype: int64

# Hide positives with unlabeled

In [53]:


mask = np.random.rand(len(X_train)) < 0.75
y_train = y_train.values
y_train[mask]=0

In [54]:


maskv = np.random.rand(len(X_val)) < 0.75
y_val = y_val
y_val[maskv]=0

In [55]:
pd.Series(y_train).value_counts()

0    102371
1      8066
dtype: int64

In [93]:
pd.Series(y_test).value_counts()

0    9672
1    4133
Name: legitimate, dtype: int64

In [81]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(X_train, y_train)

C:\Users\drdre\.conda\envs\tensorflowcpu_20220413\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [82]:
#%%
from sklearn.metrics import confusion_matrix, classification_report,f1_score,accuracy_score,roc_auc_score

In [83]:
y_pred=clf.predict(X_test)

In [84]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      1.00      0.82      9672
           1       1.00      0.00      0.01      4133

    accuracy                           0.70     13805
   macro avg       0.85      0.50      0.42     13805
weighted avg       0.79      0.70      0.58     13805



In [85]:
f1_score(y_test, y_pred),accuracy_score(y_test, y_pred)

(0.0057901085645355845, 0.7014849692140529)

In [86]:
confusion_matrix(y_test, y_pred)

array([[9672,    0],
       [4121,   12]], dtype=int64)

In [87]:
y_probsv=clf.predict_proba(X_val)[:,1]

In [88]:
y_probs=clf.predict_proba(X_test)[:,1]

In [89]:


# Function to find the optimal threshold
def find_optimal_threshold(y_true, y_probs):
    thresholds = np.arange(0.0, 1.0, 0.01)
    f1_scores = [f1_score(y_true, y_probs >= t) for t in thresholds]
    optimal_threshold = thresholds[np.argmax(f1_scores)]
    return optimal_threshold, max(f1_scores)

# Find the optimal threshold
optimal_threshold, best_f1_score = find_optimal_threshold(y_val, y_probsv)
print("Optimal threshold:", optimal_threshold)
#print("Best F1 score:", best_f1_score)

# Apply the optimal threshold to make final predictions
y_preds = (y_probs >= optimal_threshold).astype(int)

Optimal threshold: 0.09


In [90]:
f1_score(y_test,y_preds)

0.8896851804453545

In [94]:
accuracy_score(y_test,y_preds)

0.9375588554871424

In [91]:
y_probs=clf.predict_proba(X_test)[:,1]

In [92]:
roc_auc_score(y_test,y_probs)

0.9848460423747453